# Iterators

FORTRAN and C require you to write loops to process arrays, and array indexing is a frequent source of bugs in programs, it's easy to get them wrong.
Iterators lets you access elements of collections easily with simple syntax, they hide house-keeping details from you.

## Numeric range

Numeric integer sequences can be generated by:

- start:stop
- start:step:stop

In [1]:
for n in 1:5
    println("$n")
end

1
2
3
4
5


In [2]:
for n ∈ 2:2:10            # ∈ is \in<tab>
    println("$n")
end

2
4
6
8
10


To create descending sequence, an explicit step using negative number is required:

In [3]:
for n ∈ 5:-1:1
    println("$n")
end

5
4
3
2
1


## Count for ever

In [4]:
for n in Iterators.countfrom(1, 2)        # count from start and step forever
    n <= 10 || break
    println(n)
end

1
3
5
7
9


## String

A string naturally acts as an iterator:

In [5]:
for c in "abcde"
    println("$c")
end

a
b
c
d
e


## Tuple

In [6]:
for e in (1, 'a', "bc", true)
    println(e)
end

1
a
bc
true


## Array

In [7]:
m = [1 4; 2 5; 3 6]

3×2 Array{Int64,2}:
 1  4
 2  5
 3  6

In [8]:
for e in m
    println(e)
end

1
2
3
4
5
6


## Dictionary

Dictionary is an unordered collection, you can iterate over it but the order is unspecified.

In [9]:
d = Dict(1 => "a", 2 => "b", 3 => "c")

Dict{Int64,String} with 3 entries:
  2 => "b"
  3 => "c"
  1 => "a"

In [10]:
for e in d              # each entry is a "pair"
    println(e)
end

2 => "b"
3 => "c"
1 => "a"


In [11]:
for (k,v) in d
    println(k, "   ", v)
end

2   b
3   c
1   a


## Set

Like dictionary, a set is an unordered collection.

In [12]:
s = Set((1, 2, 3, 3, 4))

Set{Int64} with 4 elements:
  4
  2
  3
  1

In [13]:
for e in s
    println(e)
end

4
2
3
1


## Enumerate

If you need to keep track of the counter as well as the value, use `enumerate`:

In [14]:
for (i,v) ∈ enumerate(10:-2:2)
    println("$i  $v")
end

1  10
2  8
3  6
4  4
5  2


In [15]:
for (i,v) ∈ enumerate(d)
    println("$i:  $v")
end

1:  2 => "b"
2:  3 => "c"
3:  1 => "a"


## Zip

`zip()` iterates over multiple iterators at the same time returning a tuple every iteration, it stops on the shortest iterator:

In [16]:
a = 1:5
b = 3:3:15

for v in zip(a, b, d)
    println("$v")
end

(1, 3, 2 => "b")
(2, 6, 3 => "c")
(3, 9, 1 => "a")


## Collect

**collect()** collects the result of the iterator into an array:

In [17]:
zipped = collect(zip(a, b))

5-element Array{Tuple{Int64,Int64},1}:
 (1, 3)
 (2, 6)
 (3, 9)
 (4, 12)
 (5, 15)

Notice zip() is essentially a transpose operation.
To recover the original, we transpose the result again, but we need to destructure (splat) the zip:

In [18]:
collect(zip(zipped...))     # the ... suffix destructures the 5-element array into 5 separate tuples

2-element Array{NTuple{5,Int64},1}:
 (1, 2, 3, 4, 5)
 (3, 6, 9, 12, 15)

Note splatting a large array is way slow.

## Stateful Iterator

When you use an iterator, immutable iterators restart from the beginning.
If iteration stopped early, it won't resume from where it left off.

In [19]:
a = 1:5                         # iterator

for i in a
    println("1: $i")
    i <= 2 || break             # early termination
end

for i in a                      # restarts from beginning
    println("2: $i")
end

1: 1
1: 2
1: 3
2: 1
2: 2
2: 3
2: 4
2: 5


A stateful iterator can be resumed from the same spot like other mutable iterators:

In [20]:
a = Iterators.Stateful(1:5)     # stateful iterator

println("Length of iterator is $(length(a))")

for i in a
    println("1: $i")
    i <= 2 || break             # early termination
end

println("Length of iterator is $(length(a))")

for i in a                      # resume from same spot
    println("2: $i")
end

println("Length of iterator is $(length(a))")
println("Is it empty? $(isempty(a))")

Length of iterator is 5
1: 1
1: 2
1: 3
Length of iterator is 2
2: 4
2: 5
Length of iterator is 0
Is it empty? true
